In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import medfilt
import pyfftw

plt.rcdefaults()
plt.style.use('seaborn-paper')
plt.rc('font', **{'family': 'sans-serif', 'sans-serif': ['Helvetica']})

from ttcrpy.rgrid import Grid3d

# Example 3 - Random medium

This examples illustrates raytracing capabilities for a random spatial distribution of velocity.

In [2]:
# Generate random field

nx = 256
ny = 256
nz = 128

dx = 1.0
dy = 1.0
dz = 1.0

beta = 3.5  

m = np.random.randn(ny, nx, nz)

m_fftw = pyfftw.empty_aligned((ny, nx, nz), dtype='float32')
M_fftw = pyfftw.empty_aligned((ny, nx, nz//2 + 1), dtype='complex64')
m2_fftw = pyfftw.empty_aligned((ny, nx, nz), dtype='float32')

fft_object_fw = pyfftw.FFTW(m_fftw, M_fftw, axes=(0,1,2))
fft_object_bw = pyfftw.FFTW(M_fftw, m2_fftw, axes=(0,1,2), direction='FFTW_BACKWARD')

M = pyfftw.interfaces.numpy_fft.fftn(m, axes=(0,1,2), threads=16)

m_fftw[:] = m[:]

fft_object_fw()

kx = 2.*np.pi * np.fft.fftfreq(nx, dx)
ky = 2.*np.pi * np.fft.fftfreq(ny, dy)
kz = 2.*np.pi * np.fft.fftfreq(nz, dz)

kx, ky, kz = np.meshgrid(kx, ky, kz, indexing='ij')

k = np.sqrt( kx**2 + ky**2 + kz**2 )

ind = k != 0.0

k[ind] = k[ind]**(-beta/2.)

M = M * k
m = pyfftw.interfaces.numpy_fft.ifftn( M, axes=(0,1,2), threads=16 )

m = medfilt(np.real(m))  # smooth out a bit with a 3-pts median filter

In [3]:
# coordinates of the nodes
xn = np.arange(0, (nx+1)*dx, dx)
yn = np.arange(0, (ny+1)*dy, dy)
zn = np.arange(0, (nz+1)*dz, dz)

# create grid with default values (by default, slowness is defined for cells).
# We will compute traveltimes for two sources, this will be done in parallel
# using two threads.
grid = Grid3d(xn, yn, zn, n_threads=2)

In [4]:
# make realistic values of velocity
fac = 200.0
velocity = 4000.0 + fac * m.flatten()

# Assign slowness to grid
grid.set_slowness(1.0/velocity)

# Save to VTK format to visualize
grid.to_vtk({'Velocity': velocity}, 'example3_vel')

The velocity model looks like the following.

<img src="figs/example3_model.png"  width="800"/>

Let's now perform raytracing for some arbitrary sources and receivers.

In [5]:
# Define the source location
src = np.array([[9.0, 4.3, 124.2],
                [4.1, 241.4, 119.7]])

# Define some receivers
rcv = np.array([[222.2, 3.4, 1.2],
                [206.8, 155.4, 3.3],
                [178.9, 251.1, 10.4]])

# Make src & rcv pairs

src = np.kron(src, np.ones((3, 1)))
rcv = np.kron(np.ones((2, 1)), rcv)

tt, rays = grid.raytrace(src, rcv, return_rays=True)

# Save raypaths
grid.to_vtk({'raypaths for shot no 1': rays[:3],
            'raypaths for shot no 2': rays[3:]}, 'example3_rays')

The raypaths for the first source are in green, and in blue for the second source.

<img src="figs/example3_rays.png"  width="800"/>
